# Building simulation for NET 03:
## 3x3 superblocks
This notebook summarizes the simulation construction and running for net 03

In [1]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_50.net.xml"

### Demand generation
Endogenous demand based on ACTIVITYGEN

In [2]:
## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_50.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_50.trips.rou.xml"

## Routes output files from ACTIVITYGEN trips
ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN = ".\data/activitygen-bcn_50.rou.xml"

In [ ]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987

In [ ]:
# Routing trips from ACTIVITYGEN
# CH routing algorithm is used

!duarouter --net-file {NET_FILE} \
            --route-files {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --output-file {ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN} \
            --routing-algorithm CH \
            --ignore-errors

Passing through traffic from random trips between fringe edges

In [ ]:
# creation of hourly distribution of trips
dist = [204, 205, 245, 468, 1047, 2108, 3842, 5490, 6632, 6472, 5626, 4789, 4300, 4505, 4942, 5510, 6103, 6645, 6423, 5265, 3948, 2578, 1658, 1188]
total_random_trips = 280000
factor = total_random_trips/sum(dist)
random_dist = [int(element * factor) for element in dist]

In [ ]:
# Creating iteratively random trips accordingly to the hourly demand
## CREATE RANDOM FOLDER if it does not exist
import os
if not os.path.exists('data/random'):
    os.makedirs('data/random')

for i in range(len(random_dist)):
    begin = 3600*i
    end = 3600*(i+1)
    trips = random_dist[i]
    period = (end-begin)/trips
    trip_prefix = str(str(i) + "h")
    output_folder_1 = "data/random/random_h_"
    output_folder_2_trips = "-bcn_50.trips.rou.xml"
    output_folder_2_routes = "-bcn_50.rou.xml"

    BEGIN_ARG = str("-b " + str(begin))
    END_ARG = str("-e " + str(end))
    PERIOD_ARG = str("-p" + str(period))
    OUT_TRIPS_ARG = str("-o" + output_folder_1 + str(i) + output_folder_2_trips)
    OUT_ROUTES_ARG = str("-r" + output_folder_1 + str(i) + output_folder_2_routes)
    
    # bash SUMO command for RANDOM trips
    # Adjust the installation path for SUMO
    !python "C:\Program Files (x86)\Eclipse\Sumo\tools\randomTrips.py" -n {NET_FILE} \
    --prefix={trip_prefix} \
    --fringe-factor 100 \
    {BEGIN_ARG} \
    {END_ARG} \
    {PERIOD_ARG} \
    --binomial 10 \
    "{OUT_TRIPS_ARG}" \
    --edge-permission=passenger

In [ ]:
# Merging all random routing files i|n a single file
import os
list_trips_files = []
for file in os.listdir(r"data/random/"):
    if file.endswith(".trips.rou.xml"):
        list_trips_files.append(str("data/random/" + file))
list_trips_files_str = ' '.join([str(str(elem) + ',') for elem in list_trips_files])
list_trips_files_str = list_trips_files_str.replace(" ","")

SINGLE_RANDOM_ROUTES_FILE = "data/random/random_ALL.rou.xml"

!duarouter --net-file {NET_FILE} \
--route-files {list_trips_files_str[0:-1]} \
--output-file {SINGLE_RANDOM_ROUTES_FILE} \
--routing-algorithm CH \
--ignore-errors

### Microscopic Simulation
#### Setting configuration files
A XML configuration file (**data/compar_052.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file.xml**) for results colletion.

In [3]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder
import os, time
if not os.path.exists('data/results'):
    os.makedirs('data/results')

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output data/results/overall_stats.xml'.format(scale)
    SUMMARY_ARG = '--summary data/results/summary.xml'.format(scale)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_052.sumo.cfg \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1 \
    -W
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

STARTING sumo simulation with scale 2.00
Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 1)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 33 ACT 31 BUF 3)                               
DONE sumo simulation with scale 2.00 in 04.3 minutes 

STARTING sumo simulation with scale 1.95

Step #200.00 (1ms ~= 1000.00*RT, ~48000.00UPS, vehicles TOT 61 ACT 48 BUF 1)              
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 100 ACT 60 BUF 0)                              
Step #400.00 (1ms ~= 1000.00*RT, ~65000.00UPS, vehicles TOT 130 ACT 65 BUF 0)             
Step #500.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles TOT 166 ACT 72 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~66000.00UPS, vehicles TOT 196 ACT 66 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 218 ACT 53 BUF 2)                              
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 258 ACT 75 BUF 0)                              
Step #900.0

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 1)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~31000.00UPS, vehicles TOT 33 ACT 31 BUF 3)              
Step #200.00 (1ms ~= 1000.00*RT, ~46000.00UPS, vehicles TOT 59 ACT 46 BUF 1)              
Step #300.00 (1ms ~= 1000.00*RT, ~58000.00UPS, vehicles TOT 97 ACT 58 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~61000.00UPS, vehicles TOT 126 ACT 61 BUF 0)             
Step #500.00 (2ms ~= 500.00*RT, ~33500.00UPS, vehicles TOT 161 ACT 67 BUF 0)              
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 191 ACT 63 BUF 0)                              
Step #700.00 (1ms ~= 1000.00*RT, ~52000.00UPS, vehicles TOT 212 ACT 52 BUF 2)             
Step #800.00 (1ms ~= 1000.00*RT, ~71000.00UPS, vehicles TOT 251 ACT 71 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~76000.00UPS, vehicles TOT 284 ACT 76 BUF 0)             
Step #1000.00 (1ms ~= 1000.00*RT, ~78000.00UPS, vehicles T

Loading configuration ... done.
Step #0.00 (1ms ~= 1000.00*RT, ~1000.00UPS, vehicles TOT 1 ACT 1 BUF 1)                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 32 ACT 30 BUF 3)                               
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 58 ACT 45 BUF 0)                               
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 95 ACT 58 BUF 0)                               
Step #400.00 (1ms ~= 1000.00*RT, ~61000.00UPS, vehicles TOT 123 ACT 61 BUF 0)             
Step #500.00 (1ms ~= 1000.00*RT, ~68000.00UPS, vehicles TOT 157 ACT 68 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~60000.00UPS, vehicles TOT 186 ACT 60 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 207 ACT 50 BUF 2)                              
Step #800.00 (1ms ~= 1000.00*RT, ~68000.00UPS, vehicles TOT 245 ACT 68 BUF 0)             
Step #900.00 (2ms ~= 500.00*RT, ~36500.00UPS, vehicles TOT 277 ACT 73 BUF 0)              
Step #1000.00 (1ms ~= 1000.00*RT, ~74000.00UPS, vehicles T

Loading configuration ... done.
Step #0.00 (1ms ~= 1000.00*RT, ~1000.00UPS, vehicles TOT 1 ACT 1 BUF 1)                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 31 ACT 29 BUF 3)                               
Step #200.00 (1ms ~= 1000.00*RT, ~43000.00UPS, vehicles TOT 56 ACT 43 BUF 1)              
Step #300.00 (1ms ~= 1000.00*RT, ~54000.00UPS, vehicles TOT 92 ACT 54 BUF 0)              
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 120 ACT 60 BUF 0)                              
Step #500.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles TOT 153 ACT 72 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~59000.00UPS, vehicles TOT 181 ACT 59 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 201 ACT 48 BUF 2)                              
Step #800.00 (1ms ~= 1000.00*RT, ~68000.00UPS, vehicles TOT 238 ACT 68 BUF 0)             
Step #900.00 (2ms ~= 500.00*RT, ~38000.00UPS, vehicles TOT 270 ACT 76 BUF 0)              
Step #1000.00 (2ms ~= 500.00*RT, ~35500.00UPS, vehicles TO

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 1)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 30 ACT 28 BUF 3)                               
Step #200.00 (1ms ~= 1000.00*RT, ~41000.00UPS, vehicles TOT 55 ACT 41 BUF 0)              
Step #300.00 (1ms ~= 1000.00*RT, ~54000.00UPS, vehicles TOT 90 ACT 54 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~58000.00UPS, vehicles TOT 117 ACT 58 BUF 0)             
Step #500.00 (2ms ~= 500.00*RT, ~32500.00UPS, vehicles TOT 149 ACT 65 BUF 0)              
Step #600.00 (2ms ~= 500.00*RT, ~29000.00UPS, vehicles TOT 176 ACT 58 BUF 0)              
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 196 ACT 46 BUF 2)                              
Step #800.00 (1ms ~= 1000.00*RT, ~67000.00UPS, vehicles TOT 232 ACT 67 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~72000.00UPS, vehicles TOT 262 ACT 72 BUF 0)             
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 289 ACT 72 BUF

...

Step #22700.00 (1ms ~= 1000.00*RT, ~136000.00UPS, vehicles TOT 3033 ACT 136 BUF 1)        
Step #22800.00 (2ms ~= 500.00*RT, ~69500.00UPS, vehicles TOT 3094 ACT 139 BUF 0)          
Step #22900.00 (3ms ~= 333.33*RT, ~47666.67UPS, vehicles TOT 3158 ACT 143 BUF 0)          
Step #23000.00 (3ms ~= 333.33*RT, ~47000.00UPS, vehicles TOT 3221 ACT 141 BUF 0)          
Step #23100.00 (2ms ~= 500.00*RT, ~66500.00UPS, vehicles TOT 3283 ACT 133 BUF 0)          
Step #23200.00 (2ms ~= 500.00*RT, ~68000.00UPS, vehicles TOT 3342 ACT 136 BUF 0)          
Step #23300.00 (1ms ~= 1000.00*RT, ~134000.00UPS, vehicles TOT 3405 ACT 134 BUF 0)        
Step #23400.00 (2ms ~= 500.00*RT, ~66500.00UPS, vehicles TOT 3464 ACT 133 BUF 1)          
Step #23500.00 (3ms ~= 333.33*RT, ~48666.67UPS, vehicles TOT 3526 ACT 146 BUF 0)          
Step #23600.00 (1ms ~= 1000.00*RT, ~134000.00UPS, vehicles TOT 3583 ACT 134 BUF 0)        
Step #23700.00 (2ms ~= 500.00*RT, ~66000.00UPS, vehicles TOT 3648 ACT 132 BUF 0)          

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 2 ACT 2 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 1 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 2 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 3 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 4 BUF 0)                                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 10 ACT 2 BUF 0)                                
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 11 ACT 2 BUF 0)                                
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 13 ACT 3 BUF 0)                                
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 15 ACT 3 BUF 0)                                
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 16 ACT 3 BUF 0

Step #13700.00 (0ms ?*RT. ?UPS, vehicles TOT 340 ACT 8 BUF 0)                             
Step #13800.00 (1ms ~= 1000.00*RT, ~10000.00UPS, vehicles TOT 345 ACT 10 BUF 0)           
Step #13900.00 (1ms ~= 1000.00*RT, ~10000.00UPS, vehicles TOT 350 ACT 10 BUF 0)           
Step #14000.00 (0ms ?*RT. ?UPS, vehicles TOT 354 ACT 11 BUF 0)                            
Step #14100.00 (0ms ?*RT. ?UPS, vehicles TOT 359 ACT 11 BUF 0)                            
Step #14200.00 (0ms ?*RT. ?UPS, vehicles TOT 364 ACT 10 BUF 0)                            
Step #14300.00 (1ms ~= 1000.00*RT, ~13000.00UPS, vehicles TOT 371 ACT 13 BUF 0)           
Step #14400.00 (0ms ?*RT. ?UPS, vehicles TOT 374 ACT 11 BUF 0)                            
Step #14500.00 (0ms ?*RT. ?UPS, vehicles TOT 383 ACT 12 BUF 0)                            
Step #14600.00 (0ms ?*RT. ?UPS, vehicles TOT 393 ACT 21 BUF 0)                            
Step #14700.00 (0ms ?*RT. ?UPS, vehicles TOT 403 ACT 24 BUF 0)                            

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 2 ACT 1 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 1 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 4 ACT 1 BUF 0)                                 
Step #500.00 (1ms ~= 1000.00*RT, ~2000.00UPS, vehicles TOT 5 ACT 2 BUF 0)                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 1 BUF 0)                                 
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 6 ACT 1 BUF 0)                                 
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 2 BUF 0)                                 
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 2 BUF 0)                                 
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 1 BUF 0)

Step #52000.00 (1ms ~= 1000.00*RT, ~46000.00UPS, vehicles TOT 8954 ACT 46 BUF 0)          
Step #52100.00 (0ms ?*RT. ?UPS, vehicles TOT 8978 ACT 50 BUF 0)                           
Step #52200.00 (1ms ~= 1000.00*RT, ~52000.00UPS, vehicles TOT 9000 ACT 52 BUF 0)          
Step #52300.00 (0ms ?*RT. ?UPS, vehicles TOT 9024 ACT 51 BUF 0)                           
Step #52400.00 (1ms ~= 1000.00*RT, ~46000.00UPS, vehicles TOT 9048 ACT 46 BUF 0)          
Step #52500.00 (1ms ~= 1000.00*RT, ~48000.00UPS, vehicles TOT 9072 ACT 48 BUF 0)          
Step #52600.00 (0ms ?*RT. ?UPS, vehicles TOT 9096 ACT 53 BUF 0)                           
Step #52700.00 (1ms ~= 1000.00*RT, ~57000.00UPS, vehicles TOT 9120 ACT 57 BUF 0)          
Step #52800.00 (0ms ?*RT. ?UPS, vehicles TOT 9143 ACT 49 BUF 0)                           
Step #52900.00 (1ms ~= 1000.00*RT, ~49000.00UPS, vehicles TOT 9167 ACT 49 BUF 0)          
Step #53000.00 (1ms ~= 1000.00*RT, ~50000.00UPS, vehicles TOT 9192 ACT 50 BUF 0)          

### Mesoscopic Simulations
#### Setting configuration files
A XML configuration file (**data/compar_052_meso.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file_meso.xml**) for results collection.

Three different calibrations are finally run for the mesoscopic case, by changing the headway times ($\tau values$) and other mesoscopic parameters as follows

#### Mesoscopic simulation trial01

Param:

$\tau_{ff} = 1$

$\tau_{fj} = 1.2$

$\tau_{jf} = 1.8$

$\tau_{jj} = 1.4$

--meso-tls-penalty = 2

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_01 = r'data/results_meso_trial01'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_01):
    os.makedirs(RESULTS_FOLDER_MESO_01)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_01)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_01)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_052_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 1 \
    --meso-taufj 1.2 \
    --meso-taujf 1.8 \
    --meso-taujj 1.4 \
    --meso-lane-queue \
    --meso-tls-penalty 2 \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial02

Param:

$\tau_{ff} = 2.1$

$\tau_{fj} = 0.75$

$\tau_{jf} = 0.75$

$\tau_{jj} = 0.5$

--meso-jam-threshold = -0.5

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_02 = r'data/results_meso_trial02'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_02):
    os.makedirs(RESULTS_FOLDER_MESO_02)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_02)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_02)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_052_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 2.1 \
    --meso-taufj 0.75 \
    --meso-taujf 0.75 \
    --meso-taujj 0.5 \
    --meso-lane-queue \
    --meso-jam-threshold -0.5 \
    --meso-junction-control \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial03

Params:

$\tau_{ff} = 1.13$

$\tau_{fj} = 1.13$

$\tau_{jf} = 1.73$

$\tau_{jj} = 1.4$

--meso-jam-threshold = -1

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_03 = r'data/results_meso_trial03'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_03):
    os.makedirs(RESULTS_FOLDER_MESO_03)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_03)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_03)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_052_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-junction-control \
    --meso-lane-queue \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value